In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import mpl_toolkits.mplot3d
%matplotlib inline

In [3]:
from scipy import optimize
from scipy.io import loadmat

In [4]:
! pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163688 sha256=9c7fbaf1b5b6986e9bf4968929ec35472b5997abd5933c273abaf7e0bcd9ab0b
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [5]:
from surprise import SVD, NMF, SVDpp
from surprise import Dataset
from surprise.model_selection import cross_validate

In [6]:
path = "/content/drive/My Drive/Colab Notebooks/data7"

Домашнє завдання модуля 7.
-------------------------
Пропонуємо вам познайомитись з бібліотекою surprise, котра якраз є по суті додатком до знайомої нам бібліотеки scikit-learn для тренування моделей рекомендаційних систем.

Візьміть датасет movielens і побудуйте модель матричної факторизації. У даній бібліотеці він має назву SVD. Підберіть найкращі параметри за допомогою крос-валідації, також поекспериментуйте з іншими алгоритмами розрахунків (SVD++, NMF) і оберіть той, який буде оптимальним.

Підказки як саме побудувати дану модель ви знайдете в документації до даної бібліотеки.

In [7]:
# Load the movielens-100k dataset (download it if needed).
# data = Dataset.load_builtin(os.path.join(path,'ml-100k'))
data = Dataset.load_builtin('ml-100k')


Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


#Алгоритм SVD

In [8]:
# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9337  0.9379  0.9412  0.9297  0.9358  0.9356  0.0039  
MAE (testset)     0.7342  0.7404  0.7393  0.7317  0.7391  0.7370  0.0034  
Fit time          1.50    1.48    1.52    1.45    1.56    1.50    0.04    
Test time         0.31    0.14    0.25    0.17    0.22    0.22    0.06    


{'test_rmse': array([0.93365228, 0.93789632, 0.94115866, 0.9296582 , 0.93581249]),
 'test_mae': array([0.7342444 , 0.7404142 , 0.73932152, 0.73170944, 0.73909932]),
 'fit_time': (1.4955499172210693,
  1.484433889389038,
  1.5214893817901611,
  1.4537529945373535,
  1.5617616176605225),
 'test_time': (0.3080480098724365,
  0.14345145225524902,
  0.248246431350708,
  0.17080450057983398,
  0.22186493873596191)}

9 секунд  
Середнє значення RMSE 0.9356, найкраще 0.9412

In [17]:
from surprise.model_selection import GridSearchCV

param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5)

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

0.961004489921991
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [26]:
# Use the famous SVD algorithm.
algo = SVD(n_epochs=10, lr_all=0.005, reg_all=0.4)

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9610  0.9655  0.9595  0.9633  0.9557  0.9610  0.0033  
MAE (testset)     0.7711  0.7738  0.7686  0.7702  0.7661  0.7699  0.0026  
Fit time          0.76    0.78    0.78    0.79    0.79    0.78    0.01    
Test time         0.13    0.14    0.37    0.12    0.12    0.18    0.10    


{'test_rmse': array([0.96097422, 0.96549167, 0.95945135, 0.96330316, 0.95568822]),
 'test_mae': array([0.77105483, 0.77376419, 0.76857523, 0.77021391, 0.76609183]),
 'fit_time': (0.7640471458435059,
  0.7771883010864258,
  0.7774984836578369,
  0.7925167083740234,
  0.7901716232299805),
 'test_time': (0.12561607360839844,
  0.13522577285766602,
  0.36721014976501465,
  0.12247014045715332,
  0.12488842010498047)}

6 секунд  
Середнє значення RMSE 0.9610, найкраще 0.9633

#Алгоритм SVD++

In [9]:
# Use the famous SVD++ algorithm.
algo2 = SVDpp()

# Run 5-fold cross-validation and print results.
cross_validate(algo2, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9202  0.9145  0.9229  0.9185  0.9263  0.9205  0.0040  
MAE (testset)     0.7219  0.7148  0.7263  0.7212  0.7248  0.7218  0.0040  
Fit time          31.14   30.39   28.59   28.62   28.53   29.45   1.10    
Test time         4.53    5.97    4.61    4.23    4.26    4.72    0.64    


{'test_rmse': array([0.92017583, 0.91446369, 0.92293072, 0.91852711, 0.92631848]),
 'test_mae': array([0.72187387, 0.71477508, 0.72631206, 0.72120902, 0.72483483]),
 'fit_time': (31.14117193222046,
  30.390024185180664,
  28.59249186515808,
  28.61652159690857,
  28.525364875793457),
 'test_time': (4.529642581939697,
  5.974492073059082,
  4.605161190032959,
  4.22983193397522,
  4.263015270233154)}

2 хвилини  
Середнє значення RMSE 0.9205, найкраще 0.9263

In [13]:
from surprise.model_selection import GridSearchCV

param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVDpp, param_grid, measures=["rmse", "mae"], cv=5)

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

0.9612899398509308
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [27]:
# Use the famous SVD++ algorithm.
algo2 = SVDpp(n_epochs=10, lr_all=0.005, reg_all=0.4)

# Run 5-fold cross-validation and print results.
cross_validate(algo2, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9648  0.9548  0.9612  0.9640  0.9578  0.9606  0.0038  
MAE (testset)     0.7743  0.7632  0.7706  0.7714  0.7695  0.7698  0.0037  
Fit time          14.07   14.79   14.34   14.42   13.87   14.30   0.31    
Test time         6.25    4.11    4.45    5.30    5.32    5.08    0.75    


{'test_rmse': array([0.96484136, 0.95484563, 0.96121606, 0.96402981, 0.9578332 ]),
 'test_mae': array([0.77434907, 0.76317936, 0.77061535, 0.77135327, 0.76950538]),
 'fit_time': (14.07465147972107,
  14.792272329330444,
  14.344437837600708,
  14.419158697128296,
  13.872805118560791),
 'test_time': (6.246171951293945,
  4.10970664024353,
  4.446019649505615,
  5.298256158828735,
  5.317287921905518)}

1 хвилина  
Середнє значення RMSE 0.9606, найкраще 0.9640

#Алгоритм NMF

In [12]:
# Use the famous NMF algorithm.
algo3 = NMF()

# Run 5-fold cross-validation and print results.
cross_validate(algo3, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9652  0.9687  0.9644  0.9634  0.9636  0.9650  0.0019  
MAE (testset)     0.7585  0.7607  0.7587  0.7570  0.7580  0.7586  0.0012  
Fit time          2.14    2.88    2.82    2.11    2.18    2.43    0.35    
Test time         0.15    0.18    0.22    0.11    0.23    0.18    0.04    


{'test_rmse': array([0.96518875, 0.96865905, 0.96439924, 0.96337753, 0.96359603]),
 'test_mae': array([0.75851086, 0.76073701, 0.75873654, 0.75697042, 0.75802196]),
 'fit_time': (2.135143280029297,
  2.8811676502227783,
  2.824856758117676,
  2.1069388389587402,
  2.1790106296539307),
 'test_time': (0.15068650245666504,
  0.18218183517456055,
  0.2215266227722168,
  0.11266684532165527,
  0.22866129875183105)}

14 секунд  
Середнє значення RMSE 0.9650, найкраще 0.9687

In [21]:
from surprise.model_selection import GridSearchCV

param_grid = {"n_epochs": [5, 10], "reg_pu": [0.08, 0.1], "reg_qi": [0.08, 0.1]}
gs = GridSearchCV(NMF, param_grid, measures=["rmse", "mae"], cv=5)

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

0.9765202534941579
{'n_epochs': 10, 'reg_pu': 0.1, 'reg_qi': 0.1}


In [28]:
# Use the famous NMF algorithm.
algo3 = NMF(n_epochs=10, reg_pu=0.1, reg_qi=0.1)

# Run 5-fold cross-validation and print results.
cross_validate(algo3, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9647  0.9833  0.9794  0.9742  0.9655  0.9734  0.0074  
MAE (testset)     0.7554  0.7673  0.7616  0.7597  0.7547  0.7597  0.0046  
Fit time          0.70    0.74    0.75    0.50    0.50    0.64    0.11    
Test time         0.19    0.20    0.43    0.18    0.11    0.22    0.11    


{'test_rmse': array([0.96470631, 0.98329784, 0.97943548, 0.97416814, 0.96547529]),
 'test_mae': array([0.75541684, 0.76726178, 0.76156753, 0.75969646, 0.75472935]),
 'fit_time': (0.7037234306335449,
  0.736886739730835,
  0.7455449104309082,
  0.5001795291900635,
  0.4951756000518799),
 'test_time': (0.18645381927490234,
  0.19768118858337402,
  0.43218994140625,
  0.17886662483215332,
  0.10561394691467285)}

6 секунд  
Середнє значення RMSE 0.9734, найкраще 0.9833

In [25]:
from surprise.model_selection import GridSearchCV

param_grid = {"n_epochs": [5, 10],
              "reg_pu": [0.08, 0.1], "reg_qi": [0.08, 0.1], "reg_bu": [0.04, 0.06], "reg_bi": [0.04, 0.06],
              "lr_bu": [0.007, 0.009], "lr_bi": [0.007, 0.009]}
# algo4 = NMF(biased=True)
gs = GridSearchCV(NMF, param_grid, measures=["rmse", "mae"], cv=5)

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

0.9746856348569516
{'n_epochs': 10, 'reg_pu': 0.1, 'reg_qi': 0.1, 'reg_bu': 0.04, 'reg_bi': 0.06, 'lr_bu': 0.009, 'lr_bi': 0.007}


In [29]:
# Use the famous NMF algorithm.
algo3 = NMF(n_epochs=10, reg_pu=0.1, reg_qi=0.1, reg_bu=0.04, reg_bi=0.06, lr_bu=0.009, lr_bi=0.007)

# Run 5-fold cross-validation and print results.
cross_validate(algo3, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9761  0.9857  0.9841  0.9673  0.9737  0.9774  0.0068  
MAE (testset)     0.7609  0.7670  0.7668  0.7548  0.7595  0.7618  0.0046  
Fit time          0.47    0.51    0.51    0.49    0.51    0.50    0.02    
Test time         0.13    0.11    0.29    0.11    0.16    0.16    0.07    


{'test_rmse': array([0.97605151, 0.9856907 , 0.98410663, 0.96729076, 0.97372389]),
 'test_mae': array([0.76088904, 0.7669919 , 0.76678289, 0.75483746, 0.75945186]),
 'fit_time': (0.46961498260498047,
  0.5080833435058594,
  0.5086970329284668,
  0.4935023784637451,
  0.5078620910644531),
 'test_time': (0.12567377090454102,
  0.1140434741973877,
  0.28843164443969727,
  0.11342263221740723,
  0.15980815887451172)}

4 секунди  
Середнє значення RMSE 0.9774, найкраще 0.9857

#Висновки.
З параметрами за замовчуванням найгіршим алгоритмом є SVD++ (середнє RMSE 0.9205, найкраще 0.9263),  
трохи краще - SVD (середнє RMSE 0.9356, найкраще 0.9412)  
і найкращим - NMF (середнє RMSE 0.9650, найкраще 0.9687).

Для алроритму SVD і SVD++ при підборі параметрів {'n_epochs': 10, 'reg_pu': 0.1, 'reg_qi': 0.1} значення RMSE покращилося.  
SVD (середнє 0.9610, найкраще 0.9633).  
SVD++ (середнє 0.9606, найкраще 0.9640).  
Точність алгоритмів практично зрівнялася.

Для алроритму NMF при підборі параметрів значення RMSE також покращилося:  
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4} - середнє 0.9734, найкраще 0.9833.  
{'n_epochs': 10, 'reg_pu': 0.1, 'reg_qi': 0.1, 'reg_bu': 0.04, 'reg_bi': 0.06, 'lr_bu': 0.009, 'lr_bi': 0.007} - середнє 0.9774, найкраще 0.9857.